# OMERO Image Annotation Workflow with CellPose
This notebook guides you through annotating images in OMERO using the CellPose workflow. Each step is explained clearly.

## 1. Setup and Installation
Install and import the required libraries. Make sure you have access to OMERO and the omero-annotate-ai package.

In [5]:
# Import the main package and check dependencies
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
 )

import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")
try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Not available. Install with: pip install -e .[omero]")

omero-annotate-ai version: 0.1.2
OMERO functionality: Available


## 2. Connect to OMERO
Use the widget below to connect to your OMERO server. Fill in your server details and test the connection before proceeding.

In [6]:
# Create and display the OMERO connection widget
print("OMERO Connection Setup")
conn_widget = create_omero_connection_widget()
conn_widget.display()
print("\nAfter connecting, run the next cell to continue.")

OMERO Connection Setup
Loaded configuration from connection history: root@lcows1



After connecting, run the next cell to continue.


In [8]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established. Please use the widget above to connect.")

print("OMERO connection established.")

OMERO connection established.


## 3. Configure the Annotation Workflow
Set up your annotation workflow using the widget below. Select your working directory, OMERO container, and configure annotation parameters.

In [ ]:
# Create and display the workflow widget
print("Annotation Workflow Setup")
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()
print("\nAfter completing the workflow, run the next cell.")

Annotation Workflow Setup



After completing the workflow, run the next cell.


## 4. Review and Run the Annotation Pipeline
Preview your configuration, validate your OMERO selection, and run the annotation pipeline.

#### 4.1. Configuration summary

In [14]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

print("Configuration Summary:")
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.name}")
print(f"Model: {config.ai_model.model_type}")
print(f"Channel: {config.spatial_coverage.channels}")
print(f"Output: {config.output.output_directory}")
print(f"Resume from Table: {config.workflow.resume_from_table}")
print(f"Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})")

if config.spatial_coverage.three_d:
    print("3D processing: Enabled")

Configuration Summary:
Container: project (ID: 52)
Training Set: test_20251013_130307
Model: vit_b_lm
Channel: [0]
Output: /var/home/maartenpaul/omero_annotate_ai/omero_annotations_cp2
Resume from Table: False
Read-only Mode: False
Patches: 1 per image (512×512)


#### 4.2. Convert configuration to an annotation pipeline

In [15]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"Validating {container_type} with ID {container_id}...")
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")
print(f"Found {container_type}: {container.getName()}")

Validating project with ID 52...
Found project: test


#### 4.3. Run the annotation pipeline on the files
For CellPose, this will export the images for training into the local folder.

In [ ]:
# Run the pipeline for CellPose
print("Starting annotation pipeline...")
try:
    table_id, processed_images = pipeline.run_cp_workflow()
    print("Images successfully loaded!")
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()

2025-10-13 13:03:53,547 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to '-1'
2025-10-13 13:03:53,579 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to 0
2025-10-13 13:03:53,597 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to '-1'
2025-10-13 13:03:53,633 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to 0


Starting annotation pipeline...
Loading image IDs from project 52
Found 288 image IDs


2025-10-13 13:03:53,734 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:53,735 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:53,735 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.session.uuid' to 'ca9697b6-8ba3-416f-9304-99abf2cca61c'
2025-10-13 13:03:53,736 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to '0'
2025-10-13 13:03:53,736 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to 0
2025-10-13 13:03:53,755 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:53,757 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:53,759 DEBUG [                     omero.g

Loaded 5 images (by ID)
Defining annotation schema...
Schema defined: 5 annotations
Creating OMERO tracking table...


2025-10-13 13:03:54,076 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:54,076 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:54,076 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.session.uuid' to 'ca9697b6-8ba3-416f-9304-99abf2cca61c'
2025-10-13 13:03:54,077 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to '0'
2025-10-13 13:03:54,077 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to 0
2025-10-13 13:03:54,130 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:54,132 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:54,132 DEBUG [                     omero.g

...

2025-10-13 13:03:56,698 INFO  [                           omero.gateway] (ThreadPoolExecutor-0_0) Unregistered ca9697b6-8ba3-416f-9304-99abf2cca61c/d2cf8de9-bf7f-4ef3-bda5-20426358c0d6omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.3 -p 40887 -t 60000
2025-10-13 13:03:56,762 DEBUG [                           omero.gateway] (MainThread) Deleting FileAnnotation [[863]]. Options: ['TagAnnotation', 'TermAnnotation', 'FileAnnotation']
2025-10-13 13:03:56,763 DEBUG [                           omero.gateway] (MainThread) Delete2: 
object #0 (::omero::cmd::Delete2)
{
    targetObjects = 
    {
        key = FileAnnotation
        value = 
        {
            [0] = 863
        }
    }
    childOptions = 
    {
        [0] = object #1 (::omero::cmd::graphs::ChildOption)
        {
            includeType = {}
            excludeType = 
            {
                [0] = TagAnnotation
                [1] = TermAnnotation
                [2] = FileAnnotation
            }
            includeN

Loaded 2D data shape: (512, 512)
Image data range: 0 - 4095, dtype: uint16
Saved (tifffile): /var/home/maartenpaul/omero_annotate_ai/omero_annotations_cp2/input_validation/373_0_0_0_image.tif - shape: (512, 512), range: [0-4095]
Updating OMERO table...
Deleting table: test_20251013_130307
🗑️ Deleting table: test_20251013_130307
✅ Successfully deleted table 863


2025-10-13 13:03:57,153 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:57,153 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:57,154 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.session.uuid' to 'ca9697b6-8ba3-416f-9304-99abf2cca61c'
2025-10-13 13:03:57,154 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to '0'
2025-10-13 13:03:57,154 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.group' to 0
2025-10-13 13:03:57,186 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:03:57,187 DEBUG [                     omero.gateway.utils] (MainThread) Setting 'omero.event' to 'Internal'
2025-10-13 13:03:57,187 DEBUG [                     omero.g

Created/replaced tracking table 'test_20251013_130307' with 5 units
   Container: project 52
   Table ID: 864
OMERO table updated successfully (ID: 864)
Cellpose preparation completed: 5 images saved
Images successfully loaded!


2025-10-13 13:03:59,230 DEBUG [                    omero.util.Resources] (Thread-4  ) Executing
2025-10-13 13:03:59,231 DEBUG [                    omero.util.Resources] (Thread-4  ) Checking <omero.clients.BaseClient.startKeepAlive.<locals>.Entry object at 0x7f304dbbb750>
2025-10-13 13:03:59,284 DEBUG [                    omero.util.Resources] (Thread-4  ) Sleeping 60
2025-10-13 13:04:27,096 DEBUG [                    omero.util.Resources] (Thread-6  ) Executing
2025-10-13 13:04:27,097 DEBUG [                    omero.util.Resources] (Thread-6  ) Checking <omero.clients.BaseClient.startKeepAlive.<locals>.Entry object at 0x7f304cfeb750>
2025-10-13 13:04:27,115 DEBUG [                    omero.util.Resources] (Thread-6  ) Sleeping 60
2025-10-13 13:04:49,246 DEBUG [                     omero.gateway.utils] (ThreadPoolExecutor-0_0) Setting 'omero.client.uuid' to '789d8b5e-771d-45ae-9e8e-71f5eb9e6faa'
2025-10-13 13:04:49,247 DEBUG [                     omero.gateway.utils] (ThreadPoolExecut

#### 4.4. Run the CellPose GUI to train on the images

In [ ]:
# Run CellPose and annotate the exported images
# 
# Option 1: Use CellPose GUI
# Run from terminal: pixi run cellpose
# Then open the input_training and input_validation folders
# 
# Option 2: Use CellPose programmatically (advanced)
# from cellpose import models
# model = models.Cellpose(gpu=True, model_type='cyto')
# 
# After annotation, CellPose will save masks to output_training and output_validation folders
# These will be automatically detected and uploaded in the next step

print("Ready for CellPose annotation!")
print("1. Run CellPose GUI: 'pixi run cellpose' or 'python -m cellpose'")
print("2. Open the input folders and annotate your images")
print("3. Save masks to the output folders")
print("4. Return to this notebook and run the next cell")

Ready for CellPose annotation!
1. Run CellPose GUI: 'pixi run cellpose' or 'python -m cellpose'
2. Open the input folders and annotate your images
3. Save masks to the output folders
4. Return to this notebook and run the next cell


2025-10-13 13:05:27,115 DEBUG [                    omero.util.Resources] (Thread-6  ) Executing
2025-10-13 13:05:27,116 DEBUG [                    omero.util.Resources] (Thread-6  ) Checking <omero.clients.BaseClient.startKeepAlive.<locals>.Entry object at 0x7f304cfeb750>
2025-10-13 13:05:27,131 DEBUG [                    omero.util.Resources] (Thread-6  ) Sleeping 60
2025-10-13 13:05:59,296 DEBUG [                    omero.util.Resources] (Thread-4  ) Executing
2025-10-13 13:05:59,297 DEBUG [                    omero.util.Resources] (Thread-4  ) Checking <omero.clients.BaseClient.startKeepAlive.<locals>.Entry object at 0x7f304dbbb750>
2025-10-13 13:05:59,310 DEBUG [                    omero.util.Resources] (Thread-4  ) Sleeping 60
2025-10-13 13:06:27,132 DEBUG [                    omero.util.Resources] (Thread-6  ) Executing
2025-10-13 13:06:27,133 DEBUG [                    omero.util.Resources] (Thread-6  ) Checking <omero.clients.BaseClient.startKeepAlive.<locals>.Entry object at 0

#### 4.5. Collect the annotations and upload to OMERO to connect to the original data

In [ ]:
# Check status before uploading
print("Checking annotation status...")
status = pipeline.get_annotation_status_from_disk()

print(f"\nAnnotation Status:")
print(f"  Total annotations: {status['total_annotations']}")
print(f"  Already processed: {status['processed_annotations']}")
print(f"  Pending: {status['pending_annotations']}")

print(f"\nAvailable mask files: {len(status['available_masks'])}")
for mask_info in status['available_masks']:
    upload_status = "uploaded" if mask_info['uploaded'] else "not uploaded"
    print(f"  - {mask_info['image_name']} ({mask_info['category']}): {upload_status}")

print(f"\nMissing mask files: {len(status['missing_masks'])}")
for mask_info in status['missing_masks']:
    print(f"  - {mask_info['image_name']} ({mask_info['category']})")

# Collect and upload annotations
if status['available_masks']:
    print("\n" + "="*60)
    print("Collecting CellPose annotations and uploading to OMERO...")
    print("="*60)
    
    try:
        uploaded_count, updated_config = pipeline.collect_annotations_from_disk()
        
        if uploaded_count > 0:
            print(f"\nSuccessfully processed {uploaded_count} annotations")
            
            # Show progress summary
            progress = updated_config.get_progress_summary()
            print(f"\nOverall Progress:")
            print(f"  Total units: {progress['total_units']}")
            print(f"  Completed: {progress['completed_units']}")
            print(f"  Pending: {progress['pending_units']}")
            print(f"  Progress: {progress['progress_percent']}%")
        else:
            print("\nNo new annotations found to upload")
            print("Make sure you've annotated images in CellPose and saved them to the output folders")
            
    except Exception as e:
        print(f"Error collecting annotations: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nNo mask files available yet. Annotate images in CellPose first.")

#### 4.6. Export as bioimage.io model
For details, see: https://cellpose.readthedocs.io/en/latest/models.html#sharing-models-on-bioimage-io

## 5. Cleanup (Recommended)
Close the OMERO connection when you are finished. This is good practice, especially when working with shared servers.

In [ ]:
# Close OMERO connection (optional, but recommended)
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")